# 1. Information about the submission

## 1.1 Name and number of the assignment

Task 1. Comparison Identification

## 1.2 Student name

Yurchenko Vladislav

## 1.3 Codalab user ID

v-yurchenko

# 2. Technical Report

## 2.1 Methodology 

Задача решалась с помощью дообучения моделей (fine-tuning) семейства BERT.
Подбирались более и менее мощные модели. 
Проводился подбор гиперпараметров: 
 - размер батча
 - learning-rate
 - количество эпох
 - количество слоев для разморозки
 - параметры Dropout
Для улучшения качества модели использовался подход аугментации датасета.

## 2.2 Discussion of results

Мои эксперименты:

| Checkpoint | Additional info | F1-average on dev | F1 on test |
| :- | :- | -: | -: |
| LSTM | lilaspourpre-baseline | - | 0.535994 |
| dslim/bert-base-NER | - | 0.724906 | - |
| distilbert-base-uncased | - | 0.741844 | 0.826632 |
| distilbert-base-uncased | Augment 1 word |  0.718097 | - |
| distilbert-base-uncased | Augment 3 word |  0.728500 | - |
| cointegrated/rubert-tiny2 | - | 0.612306 | |
| nsi319/distilbert-base-uncased-finetuned-app | - | 0.717692 | - |
| bert-base-uncased | - | 0.744809 | - |
| bert-base-uncased | lr=2e-5, epoch=7 |  0.755394 | 0.840702 |
| bert-base-uncased | dropout=0.5, 0.2 |  0.760007 | 0.84868 |
| **bert-base-uncased** | **dropout=0.5, 0.5, use dev set in train** |  0.760007 | **0.8515** |
| xlm-roberta-base | - | 0.654820 | - |
| liaad-srl-en_xlmr-large_report| - | 0.363216 | - |
| sberbank-ai-ruRoberta-large| - | 0.745017 | - |
| **microsoft/deberta-v3-base** | lr=5e-5, epoch=3, dropout = 0.2 | **0.806269** | **0.870761** |
| **microsoft/deberta-v3-base** | lr=5e-5, epoch=3, dropout = 0.2 + dev in train | - | **0.871298** |


В таблице приведены неиболее интересные результаты, промежуточные варианты опущены.

Общие замечания:
 - Хорошо обучается BERT-base-uncased (не мультиязычный),
 - Неожиданно в последний момент выстрелила microsoft/deberta-v3-base, которая на dev в обычном пайплайне дала +5% на f1-average, что достаточно много!
 - Модель достаточно быстро дообучается под датасет, оптимальными параметрами выглядят:
   - config.hidden_dropout_prob = 0.2
   - attention_probs_dropout_prob = 0.2
   - epoch = 3
   - learning_rate = 5e-5
   - batch_size = 16
 - В принципе можно использовать более медленный lr=2e-5, но epochs = 7
 - При увеличении размера батча до 32 качество падает.
 - Аугментация датасета не дает значимого прироста, приводит к быстрому переобучению модели.
 - Усложнение модели не дает роста в качестве модели. Берт из коробки сразу дал хорошее качество, его тюнинг улучшил показатели. Кажется, что более сложные модели находят больше Object и Aspect, но они в разметке помечены как O. 
 - В целом возникло множество вопросов к разметке.

**Датасет**
- Датасет достаточно небольшой и несбалансирован по классам. Хотелось доразметить самому. Есть очевидные ошибки в разметке dev.tsv (windows то размечается B-Object, то нет).
- На dev метрики просаживаются. На test хорошо коррелируют с train, если модель не переобучается!
- Распределение токенов - коррелирует с метриками качества на лидерборде по типам объектов - чем меньше объектов в выборке, тем ниже метрика

| Type | Count | Percent |
| :- | -: | -: |
| O | 48512 | 0.7943% |
| B-Object | 6174 | 0.1011% |
| B-Predicate | 3109 | 0.05091% |
| B-Aspect | 2069 | 0.03388% |
| I-Aspect | 591 | 0.009677% |
| I-Predicate | 427 | 0.006992%|
| I-Object | 192 | 0.003144%|


Какие идеи еще можно было проработать более детально:

**Идея 1**. Аугментация:
 - берем слова помеченные тегами в разметке
 - маскируем случайным образом от 1 до 3-х слов токеном [MASK]
 - с помощью модели bert-base-uncased восстанавливаем слово
 - записываем в датасет
Переобучение на такой аугментации возникает из-за того, что я адекватно не делил train / test, чтобы аугментированные версии одного и того же предложения не попадали в разные наборы.

**Идея 2**. Предсказания модели в ряде случаев начинались с некорректного тега - I-Object вместо B-Object. Для таких кейсов подправлена разметка функцией fix_bio_in_file

**Идея 3**. Использовать доп. информацию о pos_tags из nltk
 - B-Object относится к тегу NN
 - B-Predicate - к JJ
 - B-Aspect - к VB

**Идея 4**. Использовать KeyBERT для извлечения из предложения устойчивых словосочетаний (важных для предложения) и считать их кандидатами на разметку. Вместе с идеей 3 это может дать увеличение метрики.

**Предложение для разбора на лекции:** Не разобрался как можно переопределить работу с HuggingFace для последовательностей чтобы использовать несколько разнородных последовательностей, а не только текст. Было бы классно услышать как это делать просто через интерфейс HF.


# 3. Code

## 3.2 Requierements

In [1]:
!pip install nlpaug transformers datasets seqeval 

In [2]:
!mkdir task1/
!mkdir task1/evaluation/
!wget -O task1/train.tsv https://raw.githubusercontent.com/s-nlp/semantic-role-labelling/main/train.tsv
!wget -O task1/dev_no_answers.tsv https://raw.githubusercontent.com/s-nlp/semantic-role-labelling/main/dev_no_answers.tsv
!wget -O task1/dev.tsv https://raw.githubusercontent.com/v-yurchenko/transformers/main/task1/dev.tsv
!wget -O task1/test_no_answers.tsv https://raw.githubusercontent.com/s-nlp/semantic-role-labelling/main/test_no_answers.tsv
!wget -O evaluate_f1_partial.py https://raw.githubusercontent.com/s-nlp/semantic-role-labelling/main/evaluation/evaluate_f1_partial.py
!wget -O f1_score_partial.py https://raw.githubusercontent.com/s-nlp/semantic-role-labelling/main/evaluation/f1_score_partial.py

mkdir: cannot create directory ‘task1/’: File exists
mkdir: cannot create directory ‘task1/evaluation/’: File exists
--2023-04-25 15:18:55--  https://raw.githubusercontent.com/s-nlp/semantic-role-labelling/main/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 599430 (585K) [text/plain]
Saving to: ‘task1/train.tsv’

task1/train.tsv     100%[===================>] 585.38K  3.56MB/s    in 0.2s    

2023-04-25 15:18:56 (3.56 MB/s) - ‘task1/train.tsv’ saved [599430/599430]

--2023-04-25 15:18:56--  https://raw.githubusercontent.com/s-nlp/semantic-role-labelling/main/dev_no_answers.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.github

In [1]:
!nvidia-smi
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Tue Apr 25 15:49:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.172.01   Driver Version: 450.172.01   CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-PCIE-40GB      On   | 00000000:A2:00.0 Off |                    0 |
| N/A   49C    P0    41W / 250W |      0MiB / 40537MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [95]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm
from sklearn.metrics import f1_score

In [96]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [97]:
def read_dataset(filename, splitter="\t"):
    data = []
    sentence = []
    tags = []
    with open(filename) as f:
        for line in f:
            if not line.isspace():
                word, tag = line.split(splitter)
                sentence.append(word)
                tags.append(tag.strip())
            else:
                data.append((sentence, tags))
                sentence = []
                tags = []
    return data

In [98]:
training_data = read_dataset("task1/train.tsv")
validation_data = read_dataset("task1/dev.tsv")

In [99]:
import pandas as pd
ner_data = pd.DataFrame(training_data, columns=['tokens', 'tags'])
val_data = pd.DataFrame(validation_data, columns=['tokens', 'tags'])
ner_data.shape, val_data.shape

((2334, 2), (283, 2))

In [100]:
# val_data содержит много примеров которые не ловит train - можно попробовать сжульничать и подмешать dev
# ner_data = pd.concat([ner_data, val_data])
ner_data = ner_data.reset_index()
ner_data

,index,tokens,tags
0,0,"[also, ,, i, have, recently, discovered, advil...","[O, O, O, O, O, O, B-Object, O, O, O, B-Predic..."
1,1,"[i, have, always, heard, that, motrin, is, bet...","[O, O, O, O, O, B-Object, O, B-Predicate, O, B..."
2,2,"[when, i, was, a, figure, skater, i, injuried,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,3,"[in, a, way, ,, halloween, is, even, better, t...","[O, O, O, O, B-Object, O, O, B-Predicate, O, B..."
4,4,"[i, think, halloween, is, actually, safer, tha...","[O, O, B-Object, O, O, B-Predicate, O, O, O, B..."
...,...,...,...
2329,2329,"[isn, ', t, plastic, safer, than, wood, .]","[O, O, O, B-Object, B-Predicate, O, B-Object, O]"
2330,2330,"[and, plastic, pallets, are, safer, than, wood...","[O, B-Object, O, O, B-Predicate, O, B-Object, ..."
2331,2331,"[plastic, laminate, flooring, is, one, of, the...","[B-Object, O, B-Aspect, O, O, O, O, O, O, O, O..."
2332,2332,"[plastic, has, long, been, considered, superio...","[B-Object, O, O, O, O, B-Predicate, O, B-Objec..."


In [101]:
from collections import Counter

label_list = []
for item in ner_data['tags']:
    label_list.extend(item)

c = Counter(label_list)
for (name, num) in c.most_common(7):
    print("{}\t{}\t{:.4}%".format(name, num, num/sum(c.values())))

O	48512	0.7943%
B-Object	6174	0.1011%
B-Predicate	3109	0.05091%
B-Aspect	2069	0.03388%
I-Aspect	591	0.009677%
I-Predicate	427	0.006992%
I-Object	192	0.003144%


In [102]:
from collections import Counter

label_list = []
for item in val_data['tags']:
    label_list.extend(item)

c = Counter(label_list)
for (name, num) in c.most_common(7):
    print("{}\t{}\t{:.4}%".format(name, num, num/sum(c.values())))

O	6734	0.8052%
B-Object	781	0.09339%
B-Predicate	386	0.04616%
B-Aspect	257	0.03073%
I-Object	113	0.01351%
I-Aspect	52	0.006218%
I-Predicate	40	0.004783%


In [103]:
label_list = []
for item in ner_data['tags']:
    label_list.extend(item)
label_list = list(set(label_list))
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-Object',
 'I-Predicate',
 'I-Object',
 'B-Predicate',
 'I-Aspect',
 'B-Aspect']

In [104]:
from sklearn.model_selection import train_test_split
ner_train, ner_test = train_test_split(ner_data, test_size=0.2, random_state=1)

In [105]:
ner_train.head(3)

,index,tokens,tags
485,485,"[i, also, preferred, the, psp, controls, to, t...","[O, O, O, O, B-Object, O, O, O, O, O, O, O, O,..."
1524,1524,"[durability, solid, concrete, construction, en...","[O, O, B-Object, O, O, O, B-Predicate, O, B-As..."
366,366,"[bawalker, -, dude, you, know, ibm, has, maybe...","[O, O, O, O, O, B-Object, O, O, O, O, O, O, O,..."


In [106]:
from datasets import load_dataset, load_metric
from datasets import Dataset, DatasetDict
ner_dataset = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_dataset

DatasetDict({
    train: Dataset({
        features: ['index', 'tokens', 'tags', '__index_level_0__'],
        num_rows: 1867
    })
    test: Dataset({
        features: ['index', 'tokens', 'tags', '__index_level_0__'],
        num_rows: 467
    })
})

In [107]:
id2label = dict(enumerate(label_list))
label2id = {v: k for k, v in id2label.items()}
print(id2label, label2id)

{0: 'O', 1: 'B-Object', 2: 'I-Predicate', 3: 'I-Object', 4: 'B-Predicate', 5: 'I-Aspect', 6: 'B-Aspect'} {'O': 0, 'B-Object': 1, 'I-Predicate': 2, 'I-Object': 3, 'B-Predicate': 4, 'I-Aspect': 5, 'B-Aspect': 6}


In [108]:
batch_size = 16
from transformers import AutoTokenizer, BertForTokenClassification, DistilBertTokenizer, DistilBertModel, DistilBertForTokenClassification
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoConfig

# model_checkpoint = 'dslim/bert-base-NER'
# model_checkpoint = 'distilbert-base-uncased'
# model_checkpoint = "cointegrated/rubert-tiny2"
# model_checkpoint = 'nsi319/distilbert-base-uncased-finetuned-app'
# model_checkpoint = 'bert-base-uncased'
# model_checkpoint = 'xlm-roberta-base'
model_checkpoint = 'microsoft/deberta-v3-base'
# model_checkpoint = 'microsoft/deberta-v3-large'


result_name = model_checkpoint.replace("/", "-")

if model_checkpoint == "bert-base-uncased":
    # configuration = AutoConfig.from_pretrained(model_checkpoint)
    # print(configuration)
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    model = BertForTokenClassification.from_pretrained('bert-base-uncased', 
                                                       num_labels=len(id2label),
                                                       id2label=id2label,
                                                       label2id=label2id,
                                                       # config=configuration
                                                      )
    model.config.hidden_dropout_prob = 0.2
    model.config.attention_probs_dropout_prob = 0.2
    
else:    
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=False)
    model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, 
                                                            num_labels=len(label_list),
                                                            ignore_mismatched_sizes=True,
                                                            id2label=id2label,
                                                            label2id=label2id
                                                            )
    model.config.hidden_dropout_prob = 0.2
    model.config.attention_probs_dropout_prob = 0.2


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/jovyan/.imgenv-beautiful-curie-0/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:455: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForTokenClassification: ['mask_predictions.

In [109]:
tokenized_input = tokenizer(ner_dataset['test']["tokens"][10], is_split_into_words=True,  return_offsets_mapping=True, )
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
if model_checkpoint == 'xlm-roberta-base' or model_checkpoint == 'microsoft/deberta-v3-base' or model_checkpoint == 'microsoft/deberta-v3-large' :
    PRE_WORD = '_'
else:
    PRE_WORD = '##'
# SOS_TOKEN = "[CLS]"
# EOS_TOKEN = "[SEP]"
print(tokenized_input)
print(tokens)
print(PRE_WORD)

{'input_ids': [1, 2032, 289, 8405, 372, 1229, 322, 493, 354, 1847, 335, 4062, 264, 2894, 366, 2450, 366, 263, 459, 3909, 265, 2119, 323, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 5), (0, 2), (0, 6), (0, 3), (0, 4), (0, 2), (0, 6), (0, 4), (0, 4), (0, 4), (0, 7), (0, 2), (0, 4), (0, 1), (0, 4), (0, 1), (0, 3), (0, 4), (0, 7), (0, 2), (0, 3), (0, 1), (0, 0)]}
['[CLS]', '▁metal', '▁or', '▁cement', '▁may', '▁hold', '▁up', '▁better', '▁than', '▁wood', '▁when', '▁exposed', '▁to', '▁rain', '▁,', '▁wind', '▁,', '▁and', '▁high', '▁amounts', '▁of', '▁sun', '▁.', '[SEP]']
_


In [110]:
def tokenize_and_align_labels(examples, label_all_tokens=False):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [111]:
tokenized_datasets = ner_dataset.map(tokenize_and_align_labels, batched=True)
# tokenized_datasets['train'][0]

Map:   0%|          | 0/1867 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/467 [00:00<?, ? examples/s]

In [112]:
batch_size = 16

args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=7,
    warmup_steps=500, 
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_inputs_for_metrics=True,
)

In [113]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [114]:
metric = load_metric("seqeval")

In [115]:
example = ner_dataset['train'][4]
labels = example['tags']
metric.compute(predictions=[labels], references=[labels])

{'Aspect': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'Object': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'Predicate': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [116]:
import numpy as np

def compute_metrics(p):
    predictions, labels, inputs = p.predictions, p.label_ids, p.inputs
    predictions = np.argmax(p.predictions, axis=2)

    # send only the first token of each word to the evaluation
    true_predictions = []
    true_labels = []
    for prediction, label, tokens in zip(predictions, labels, inputs):
        true_predictions.append([])
        true_labels.append([])
        # print(prediction, label, tokens)
        for (p, l, t) in zip(prediction, label, tokens):
            # print(l, p, tokenizer.convert_ids_to_tokens(int(t)))
            if l != -100 and not tokenizer.convert_ids_to_tokens(int(t)).startswith(PRE_WORD):
                # print('append')
                true_predictions[-1].append(label_list[p])
                true_labels[-1].append(label_list[l])
    # print(true_predictions, true_labels)

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [117]:
# разморозка
# for layer in model.roberta.encoder.layer[:freeze_layer_count]:
NUM_TO_FREEZE = 0

if NUM_TO_FREEZE == 0:
    for param in model.parameters():
        param.requires_grad = True
else:
    for layer in model.bert.encoder.layer[:NUM_TO_FREEZE]:
        for param in layer.parameters():
            param.requires_grad = False
    for layer in model.bert.encoder.layer[NUM_TO_FREEZE:]:
        for param in model.parameters():
            param.requires_grad = True

In [118]:
batch_size = 16
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    warmup_steps=200, 
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_inputs_for_metrics=True,
)

In [119]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [120]:
trainer.train()

/home/jovyan/.imgenv-beautiful-curie-0/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.200134,0.774306,0.886093,0.826436,0.933106
2,No log,0.163655,0.829684,0.903311,0.864933,0.946185
3,No log,0.155860,0.865360,0.896689,0.880746,0.951849


TrainOutput(global_step=351, training_loss=0.3143968500642695, metrics={'train_runtime': 45.2582, 'train_samples_per_second': 123.757, 'train_steps_per_second': 7.756, 'total_flos': 156359765566554.0, 'train_loss': 0.3143968500642695, 'epoch': 3.0})

In [121]:
trainer.evaluate()

{'eval_loss': 0.1558600664138794,
 'eval_precision': 0.8653600340860673,
 'eval_recall': 0.8966887417218543,
 'eval_f1': 0.8807458803122291,
 'eval_accuracy': 0.9518493835388204,
 'eval_runtime': 0.993,
 'eval_samples_per_second': 470.296,
 'eval_steps_per_second': 30.212,
 'epoch': 3.0}

In [122]:
trainer.save_model(f'task1/models/{result_name}')

In [123]:
model = AutoModelForTokenClassification.from_pretrained(f"task1/models/{result_name}").to('cuda:0')

In [124]:
DEV = True
if DEV:
    test_data = read_dataset("task1/dev.tsv")#, splitter="\n")
else:
    test_data = read_dataset("task1/test_no_answers.tsv", splitter="\n")

In [125]:
def predict_sentence(sentence):
    inputs = tokenizer(sentence,
                        is_split_into_words=True, 
                        return_offsets_mapping=True, 
                        padding='max_length', 
                        truncation=True, 
                        return_tensors="pt")
    # move to gpu
    ids = inputs["input_ids"].to('cuda:0')
    mask = inputs["attention_mask"].to('cuda:0')
    # forward pass
    outputs = model(ids, attention_mask=mask)
    logits = outputs[0]

    active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [label_list[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    prediction = []
    # print(inputs["offset_mapping"].squeeze().tolist())
    for token_pred, mapping, logs in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist(), active_logits):
        # print(torch.nn.functional.sigmoid(logs))
        # only predictions on first word pieces are important
        if mapping[0] == 0 and mapping[1] != 0:
            prediction.append(token_pred[1])
        else:
            continue
    return prediction

# sentence = ner_dataset['train'][15]['tokens'] #"@HuggingFace is a company based in New York, but is also has employees working in Paris"
# print(sentence)
# print(predict_sentence(sentence))

print(test_data[2][0])
print(predict_sentence(test_data[2][0]))

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.


['i', 'have', 'tried', 'windows', '8', 'and', 'it', "'", 's', 'lighter', 'than', 'windows', 'xp', '.']
['O', 'O', 'O', 'B-Object', 'O', 'O', 'O', 'O', 'O', 'B-Predicate', 'O', 'B-Object', 'B-Object', 'O']


In [126]:
with open(f"task1/{result_name}.tsv", "w") as f:
    with torch.no_grad():
        for sentence in tqdm(test_data):
            prediction = predict_sentence(sentence[0])
            # prediction = predict_w_keyword(sentence[0])            
            for w,t in zip(sentence[0], prediction):
                # print(w, '\t', t)
                f.write(w+'\t'+t+'\n')
            f.write('\n')


100%|██████████| 283/283 [00:05<00:00, 53.34it/s]


In [127]:
# Корректируем BIO разметку на предсказании если модель выдает некорректно I вперед B и отдельно стоящую I в B
def fix_bio_in_file(fn, fnout):
    words = []
    tags = []
    prev_tags = []
    with open(fn, "r") as f:
        for line in f:
            line = line.strip("\r\n").split("\t")
            if len(line) > 1:
                words.append(line[0])
                tags.append(line[1])
            else:
                words.append(line[0])
                tags.append('')

    with open(fnout, "w") as f:
        prev_tags = tags.copy()
        prev_tags.insert(0, 'O')
        # print(prev_tags)
        for w, t, p in zip(words, tags, prev_tags):
            # print(w,t,p)
            if p.startswith('O') and t.startswith('I'):
                print("correct I->B")
                print(w, t, p)
                t = 'B' + t[1:]
                print(w, t, p)
            if t.startswith('B') and t == p:
                print("correct B->I")
                t = 'I' + t[1:]
            if w == '':
                f.write('\n')
            else:
                f.write(w + '\t'+ t + '\n')

fix_bio_in_file(f"task1/{result_name}.tsv", f"task1/{result_name}-fix.tsv")             

correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct B->I
correct I->B
receiver I-Aspect O
receiver B-Aspect O
correct I->B
poverty I-Aspect O
poverty B-Aspect O
correct I->B
income I-Aspect O
income B-Aspect O
correct I->B
of I-Aspect O
of B-Aspect O
correct I->B
fast I-Predicate O
fast B-Predicate O
correct B->I
correct I->B
timber I-Aspect O
timber B-Aspect O
correct B->I
correct I->B
glass I-Object O
glass B-Object O
correct B->I
correct I->B
sale I-Aspect O
sale B-Aspect O
correct I->B
apparel I-Aspect O
apparel B-Aspect O
correct B->I


In [128]:
from evaluate_f1_partial import main

In [129]:
main('task1/dev.tsv', f'task1/{result_name}-fix.tsv', f'task1/{result_name}_report-no-fix.txt')

In [92]:
from zipfile import ZipFile

In [93]:
with ZipFile(f'task1/{result_name}-fix.zip', 'w') as zipObj2:
   # Add multiple files to the zip
   zipObj2.write(f'task1/{result_name}-fix.tsv', arcname=f'{result_name}-fix.tsv')

## 3.2. Augmentation

In [39]:
from transformers import pipeline

def create_pipeline(model_name):
    return pipeline('fill-mask', model=model_name, device=0)

predict_mask = create_pipeline("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [40]:
import numpy as np
import itertools

def augment_sent_3_token(s, tag):
    bio = [i for i, s in enumerate(tag) if not s.startswith('O')]
    sent = s.copy()
    slen = len(sent)

    ch = np.random.choice(bio, 3,  replace=False)
    ch = sorted(ch)
    # print(ch)
    masked_words = []
    for c in ch:
        masked_words.append(sent[c])
        sent[c] = '[MASK]'
    # print(sent)
    # print(masked_words)
    result = predict_mask(" ".join(sent))  

    res_words = []
    for idx, mask in enumerate(result):
        p = []
        for pos in mask:
            if pos['score'] > 0.10:
                # print(pos)
                p.append(pos['token_str'])
        res_words.append(p)
        # print('---')
    # print(res_words)
    
    ret = []
    for pairs in itertools.product(res_words[0], res_words[1], res_words[2]):
        for idx, w in enumerate(pairs):
            sent[ch[idx]] = w
            # print(w)
        ret.append(sent.copy())
        # print(' '.join(s))
    return ret

pos = 22
augs = augment_sent_3_token(ner_data['tokens'][pos], ner_data['tags'][pos])
print(ner_data['tokens'][pos],'\n')
for au in augs:
    print(au)

['i', 'can', "'", 't', 'understand', 'why', 'they', "'", 'd', 'turn', 'down', 'the', 'mit', 'engineer', 'who', 'is', 'probably', 'much', 'smarter', 'then', 'the', 'harvard', 'poetry', 'major', '.'] 



In [41]:
import nlpaug.augmenter.word as naw

# решил не использовать, так как часто аугментация получается невыровненная. подойдет для классификации, а не для NER
def augment_sent_nlpaug(sent):
    text = " ".join(sent)
    aug = naw.ContextualWordEmbsAug(
        model_path='bert-base-uncased', action="substitute")
    augmented_text = aug.augment(text)
    return [sent.split(" ") for sent in augmented_text]
    

pos = 24
augs = augment_sent_nlpaug(ner_data['tokens'][pos]) 
print(ner_data['tokens'][pos])
print(augs)

['it', 'is', 'even', 'harder', 'to', 'get', 'off', 'the', 'waitlist', 'from', 'mit', 'than', 'at', 'harvard', 'or', 'stanford', '.']
[['harvard', 'is', 'even', 'easier', 'for', 'get', 'off', 'the', 'waitlist', 'from', 'mit', 'then', 'at', 'yale', 'or', 'rutgers.']]


In [42]:
import numpy as np

def augment_sent(s, tag):
    bio = [i for i, s in enumerate(tag) if not s.startswith('O')]
    augmented = []
    for mask_token in bio:
        sent = s.copy()
        masked_word = sent[mask_token] 
        sent[mask_token] = '[MASK]'
        # print(masked_word)
        result = predict_mask(" ".join(sent))
        for r in result:
            if r['token_str'] != masked_word and r['score'] > 0.15:
                # print(r)
                sent[mask_token] = r['token_str'].replace("#", "")
                augmented.append(sent)
    return augmented

pos = 15
augs = augment_sent(ner_data['tokens'][pos], ner_data['tags'][pos])
print(ner_data['tokens'][pos],'\n')
for au in augs:
    print(au)
# for sent in ner_data['tokens']:
#     print(sent, '\n', augment_sent(sent), '\n\n')

# predict_mask("men should wear shirt and [MASK] for tomorrow's event.")

['this', 'fatality', 'rate', 'makes', 'skiing', 'and', 'snowboarding', 'more', 'dangerous', 'than', 'bicycling', 'and', 'slightly', 'safer', 'than', 'swimming', '.'] 

['this', 'fatality', 'rate', 'makes', 'skiing', 'and', 'hiking', 'more', 'dangerous', 'than', 'bicycling', 'and', 'slightly', 'safer', 'than', 'swimming', '.']
['this', 'fatality', 'rate', 'makes', 'skiing', 'and', 'snowboarding', 'less', 'dangerous', 'than', 'bicycling', 'and', 'slightly', 'safer', 'than', 'swimming', '.']
['this', 'fatality', 'rate', 'makes', 'skiing', 'and', 'snowboarding', 'more', 'dangerous', 'than', 'hiking', 'and', 'slightly', 'safer', 'than', 'swimming', '.']
['this', 'fatality', 'rate', 'makes', 'skiing', 'and', 'snowboarding', 'more', 'dangerous', 'than', 'bicycling', 'and', 'slightly', 'safer', 'than', 'cycling', '.']
['this', 'fatality', 'rate', 'makes', 'skiing', 'and', 'snowboarding', 'more', 'dangerous', 'than', 'bicycling', 'and', 'slightly', 'safer', 'than', 'cycling', '.']


In [43]:
from tqdm import tqdm

training_data_2 = []
t = tqdm(total = len(ner_data['tokens']))
for idx, (sent, tags) in enumerate(zip(ner_data['tokens'], ner_data['tags'])):
    aug_sents = augment_sent_3_token(sent, tags)
    for i, aug_sent in enumerate(aug_sents):
        training_data_2.append((aug_sent, tags))
    aug_sents = augment_sent(sent, tags)
    for i, aug_sent in enumerate(aug_sents):
        training_data_2.append((aug_sent, tags))
    # if idx > 10:
    #     break
    t.update(1)
t.close()
         
ner_data_aug = pd.DataFrame(training_data_2, columns=['tokens', 'tags'])
ner_data = pd.concat([ner_data, ner_data_aug])
ner_data

  0%|          | 0/2334 [00:00<?, ?it/s]/home/user/conda/lib/python3.7/site-packages/transformers/pipelines/base.py:1073: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
100%|██████████| 2334/2334 [02:28<00:00, 15.70it/s]


,index,tokens,tags
0,0.0,"[also, ,, i, have, recently, discovered, advil...","[O, O, O, O, O, O, B-Object, O, O, O, B-Predic..."
1,1.0,"[i, have, always, heard, that, motrin, is, bet...","[O, O, O, O, O, B-Object, O, B-Predicate, O, B..."
2,2.0,"[when, i, was, a, figure, skater, i, injuried,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,3.0,"[in, a, way, ,, halloween, is, even, better, t...","[O, O, O, O, B-Object, O, O, B-Predicate, O, B..."
4,4.0,"[i, think, halloween, is, actually, safer, tha...","[O, O, B-Object, O, O, B-Predicate, O, O, O, B..."
...,...,...,...
20394,NaN,"[in, the, content, of, search, engines, ,, bin...","[O, O, O, O, B-Aspect, I-Aspect, O, B-Object, ..."
20395,NaN,"[in, the, content, of, search, results, ,, it,...","[O, O, O, O, B-Aspect, I-Aspect, O, B-Object, ..."
20396,NaN,"[in, the, content, of, search, results, ,, bin...","[O, O, O, O, B-Aspect, I-Aspect, O, B-Object, ..."
20397,NaN,"[in, the, content, of, search, results, ,, bin...","[O, O, O, O, B-Aspect, I-Aspect, O, B-Object, ..."


In [44]:
ner_data['str'] = ner_data['tokens'].apply(lambda x: " ".join(x))
val_data['str'] = val_data['tokens'].apply(lambda x: " ".join(x))

In [45]:
ner_data = ner_data.drop_duplicates(subset=['str'])
ner_data.shape

(19879, 4)

In [46]:
ner_data.reset_index().to_csv('task1/augmented.tsv', sep='\t', index=None)

In [47]:
import pandas as pd
ner_data = pd.read_csv('task1/augmented.tsv', sep='\t', index_col=None)

In [48]:
# с этой точки можно вернуться и попробовать переобучить БЕРТ
ner_data.tokens = ner_data.tokens.apply(eval)
ner_data.tags = ner_data.tags.apply(eval)

In [50]:
ner_data.shape

(19879, 5)

## 3.3. Идея с усреднением вероятностей предсказания класса от 3-х моделей
не взлетела

In [134]:
def predict_sentence_w_probs(sentence):
    inputs = tokenizer(sentence,
                        is_split_into_words=True, 
                        return_offsets_mapping=True, 
                        padding='max_length', 
                        truncation=True, 
                        return_tensors="pt")
    # move to gpu
    ids = inputs["input_ids"].to('cuda:0')
    mask = inputs["attention_mask"].to('cuda:0')
    # forward pass
    outputs = model(ids, attention_mask=mask)
    logits = outputs[0]

    active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [label_list[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    prediction = []
    probs = []
    # print(inputs["offset_mapping"].squeeze().tolist())
    for token_pred, mapping, logs in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist(), 
                                         nn.functional.sigmoid(active_logits).detach().cpu().numpy()):
        # print(torch.nn.functional.sigmoid(logs))
      #only predictions on first word pieces are important
        if mapping[0] == 0 and mapping[1] != 0:
            prediction.append(token_pred[1])
            probs.append(logs)
        else:
            continue
    return prediction, probs

In [135]:
with open(f"task1/{result_name}-logs.tsv", "w") as f:
    with torch.no_grad():
        for sentence in tqdm(test_data):
            # print(sentence[0])
            prediction, probs = predict_sentence_w_probs(sentence[0])
            for w,t,p in zip(sentence[0], prediction, probs):
                # print(w, '\t', t)
                f.write(w+'\t'+t+'\t'+'\t'.join(map(str, p)) +'\n')
            f.write('\n')


  0%|          | 0/283 [00:00<?, ?it/s]/home/user/conda/lib/python3.7/site-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 283/283 [00:02<00:00, 130.12it/s]


In [ ]:
# идея взять и дообучиться на вероятностях от 3-х моделей. 
# не успел по времени 
logits_files = ['dslim-bert-base-NER-logs.tsv',
                'cointegrated-rubert-tiny2-logs.tsv',
                'bert-base-uncased-logs.tsv']

match = []
for lf in logits_files:
    with open('task1/'+lf, 'r') as f:
        match.append(f.readlines())

with open('task1/mean-prob.tsv', 'w') as f:        
    for e1, e2, e3 in zip(match[0], match[1], match[2]):
        r1 = e1.strip('\n').split("\t")
        r2 = e2.strip('\n').split("\t")
        r3 = e3.strip('\n').split("\t")
        if len(r1) > 2:
            score1 = r1[2:]
            score2 = r2[2:]
            score3 = r3[2:]
            res_score = [np.mean([float(s1), float(s2), float(s3)]) for s1, s2, s3 in zip(score1, score2, score3)]
            # print(r1[:2], id2label[np.argmax(res_score)])
            f.write(r1[0]+'\t'+id2label[np.argmax(res_score)]+'\n')
        else:
            f.write('\n')
            pass
    
    

In [ ]:
# не взлетела - нужно не усреднять выходы, а добавлять знания из других источников
fix_bio_in_file(f"task1/mean-prob.tsv", f"task1/mean-prob-fix.tsv")       
main('task1/dev.tsv', f'task1/mean-prob-fix.tsv', f'task1/mean-prob_report.txt')

## 3.4 Набросок идеи про голосование 5 моделей 

In [149]:
# набросок идеи, чтоб просто использовать результаты от 5 моделей, и использовать 
def read_res_file(fn):
    words = []
    tags = []
    with open(fn, "r") as f:
        for line in f:
            line = line.strip("\r\n").split("\t")
            if len(line) > 1:
                words.append(line[0])
                tags.append(line[1])
            else:
                words.append(line[0])
                tags.append('')
    return words, tags
    
    
words, tags1 = read_res_file("task1/bert-base-uncased-fix.tsv")
words, tags2 = read_res_file("task1/nsi319-distilbert-base-uncased-finetuned-app-fix.tsv")
words, tags3 = read_res_file("task1/distilbert-base-uncased-fix.tsv")
words, tags4 = read_res_file("task1/dslim-bert-base-NER-fix.tsv")
words, tags5 = read_res_file("task1/xlm-roberta-base-fix.tsv")

In [175]:
with open("task1/majority-class.tsv", "w") as f:
    for w, t1, t2, t3, t4, t5 in zip(words, tags1, tags2, tags3, tags4, tags5):
        if w != '':
            x = [label2id[t1], label2id[t2], label2id[t3],label2id[t4],label2id[t5]]
            # выбираем метку класса, который встречается чаще других
            unique, counts = np.unique(x, return_counts=True)
            a = [(u, c) for u, c in zip(unique, counts)]
            a = sorted(a, key= lambda x: x[1])
            token_win =  a[-1][0]
            # print(w, token_win, label_list[token_win])
            # token_win = max(label2id[t1], label2id[t2], label2id[t3],label2id[t4],label2id[t5],)
            # print(w, t1, t2, t3, t4, t5, '-->', label_list[token_win])
            f.write(w+"\t"+label_list[token_win]+"\n")
        else:
            f.write("\n")
            # print('empty line')

In [ ]:
# не взлетела - нужно не усреднять выходы, а добавлять знания из других источников
fix_bio_in_file(f"task1/majority-class.tsv", f"task1/majority-class-fix.tsv")       
main('task1/dev.tsv', f'task1/majority-class-fix.tsv', f'task1/majority-class_report.txt')

## 3.5. Использовать информацию о ключевых словах для доразметки

In [86]:
!pip install keybert keybert[flair] nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Using cached keybert-0.7.0-py3-none-any.whl
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached rich-13.3.4-py3-none-any.whl (238 kB)
  Using cached flair-0.12.2-py3-none-any.whl (373 kB)
  Using cached mpld3-0.3-py3-none-any.whl
  Using cached sqlitedict-2.1.0-py3-none-any.whl
  Using cached segtok-1.5.11-py3-none-any.whl (24 kB)
  Using cached Wikipedia_API-0.5.8-py3-none-any.whl (13 kB)
  Using cached gensim-4.2.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (24.1 MB)
  Using cached ftfy-6.1.1-py3-none-any.whl (53 kB)
  Using cached langdetect-1.0.9-py3-none-any.whl
  Using cached bpemb-0.3.4-py3-none-any.whl (19 kB)
  Using ca

In [82]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [126]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
from flair.embeddings import TransformerDocumentEmbeddings
roberta = TransformerDocumentEmbeddings('roberta-base')
kw_model = KeyBERT(model=roberta)

def get_kw_vector(sent):
    keywords = kw_model.extract_keywords(sent, keyphrase_ngram_range=(1, 2))
    kw = [1 if len(k) > 0 and k[0][1] > 0.5 else 0 for k in keywords]
    return kw

In [136]:
ner_data.head(3)

,index,tokens,tags,pos,kw
0,0,"[also, ,, i, have, recently, discovered, advil...","[O, O, O, O, O, O, B-Object, O, O, O, B-Predic...","[RB, ,, NN, VB, RB, VB, NN, NN, VB, RB, JJ, CC...","[0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, ..."
1,1,"[i, have, always, heard, that, motrin, is, bet...","[O, O, O, O, O, B-Object, O, B-Predicate, O, B...","[NN, VB, RB, VB, IN, NN, VB, JJ, IN, NN, IN, N...","[0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, ..."
2,2,"[when, i, was, a, figure, skater, i, injuried,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[WR, NN, VB, DT, NN, NN, NN, VB, PR, NN, PD, D...","[0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, ..."


In [125]:
ner_data['pos'] = ner_data['tokens'].apply(lambda x: [tag[:2] for w, tag in nltk.pos_tag(x)])
val_data['pos'] = val_data['tokens'].apply(lambda x: [tag[:2] for w, tag in nltk.pos_tag(x)])

In [131]:
ner_data['kw'] = ner_data['tokens'].apply(lambda x: get_kw_vector(x))
val_data['kw'] = val_data['tokens'].apply(lambda x: get_kw_vector(x))

In [129]:
pos = 1
kw = get_kw_vector(val_data['tokens'][pos])
tab = []
for w, t, p, k in zip(val_data['tokens'][pos], val_data['tags'][pos], val_data['pos'][pos], kw):
    tab.append((w, t, p, k))
pd.DataFrame.from_records(tab)
# вопросы к разметке dev.tsv -- 
# почему в части случаев windows то размечена как B-Object, то не размечена?

,0,1,2,3
0,windows,O,NN,1
1,7,O,CD,0
2,is,O,VB,0
3,still,O,RB,0
4,going,O,VB,1
5,strong,O,JJ,1
6,even,O,RB,0
7,though,O,IN,0
8,the,O,DT,0
9,day,O,NN,1


In [135]:
pos_list = []
for item in ner_data['pos']:
    pos_list.extend(item)
id2pos = dict(enumerate(set(pos_list)))
pos2id = {v: k for k, v in id2pos.items()}
print(id2pos, pos2id)

{0: 'WP', 1: 'WD', 2: "''", 3: 'RP', 4: 'MD', 5: 'PR', 6: '.', 7: 'LS', 8: 'EX', 9: 'SY', 10: 'CD', 11: 'WR', 12: ',', 13: '$', 14: 'NN', 15: 'TO', 16: 'JJ', 17: 'VB', 18: 'DT', 19: 'PD', 20: 'PO', 21: '(', 22: 'FW', 23: '#', 24: 'CC', 25: 'IN', 26: ')', 27: 'RB', 28: ':', 29: 'UH'} {'WP': 0, 'WD': 1, "''": 2, 'RP': 3, 'MD': 4, 'PR': 5, '.': 6, 'LS': 7, 'EX': 8, 'SY': 9, 'CD': 10, 'WR': 11, ',': 12, '$': 13, 'NN': 14, 'TO': 15, 'JJ': 16, 'VB': 17, 'DT': 18, 'PD': 19, 'PO': 20, '(': 21, 'FW': 22, '#': 23, 'CC': 24, 'IN': 25, ')': 26, 'RB': 27, ':': 28, 'UH': 29}


In [146]:
from sklearn.model_selection import train_test_split
ner_train, ner_test = train_test_split(ner_data, test_size=0.2, random_state=1)

In [233]:
ner_dataset = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_dataset

DatasetDict({
    train: Dataset({
        features: ['index', 'tokens', 'tags', 'pos', 'kw', '__index_level_0__'],
        num_rows: 1867
    })
    test: Dataset({
        features: ['index', 'tokens', 'tags', 'pos', 'kw', '__index_level_0__'],
        num_rows: 467
    })
})

In [234]:
# попытка добавить данные к нашему датасету
def tokenize_and_align_labels_w_sources(examples, label_all_tokens=False):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    
    pos = []
    for p in examples["pos"]:
        pos.append([pos2id[i] for i in p])
    tokenized_inputs["posid"] = pos
    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [235]:
tokenized_datasets = ner_dataset.map(tokenize_and_align_labels_w_sources, batched=True)
# tokenized_datasets['train'][0]

Map:   0%|          | 0/1867 [00:00<?, ? examples/s]

Map:   0%|          | 0/467 [00:00<?, ? examples/s]

In [240]:
### Вот отсюда у меня не получилось понять как работать в парадигме HuggingFace с несколькими источниками
## последовательных данных. Было бы круто, если бы кто-то показал как это делать - более элегантно, чем
## как добавить тут данные?

from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer, padding=True)
train_dataloader = DataLoader(tokenized_datasets['train'], shuffle=True, batch_size=16, 
collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_datasets['test'], batch_size=16, collate_fn=data_collator)
for epoch in range(2):
    model.train()
    for step, batch in enumerate(train_dataloader):          
          outputs = model(**batch)
          loss = outputs.loss

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`tokens` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer)

In [226]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [227]:
trainer.evaluate()

{'eval_loss': 2.0239012241363525,
 'eval_precision': 0.01285140562248996,
 'eval_recall': 0.04944812362030905,
 'eval_f1': 0.02040072859744991,
 'eval_accuracy': 0.07514161946017994,
 'eval_runtime': 0.7209,
 'eval_samples_per_second': 647.765,
 'eval_steps_per_second': 41.612}

In [228]:
for idx, batch in enumerate(trainer.get_train_dataloader()):
    print(batch)
        if idx > 0:
            break

{'input_ids': tensor([[  101,  2065,  1996,  2091, 24388,  2063,  3681,  1013,  5894,  1013,
          2723,  2024, 10612,  2007,  3886,  2023,  2003,  6082,  1998, 16269,
          2084,  3796,  2030,  5509,  1012,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [  101,  2348,  2045,  2024,  2053, 15764,  2529,  2913,  2055,  1996,
          3896,  1997,  5948,  5572,  2005,  3096,  2740,  1010,  5572,  1011,
          2665,  1010,  2317,  1010,  2030,  2304,  1010, 24689,  7959, 15833,
          2030, 11703, 10354,  7959, 15833,  1011,  2003,  2467,  1037,  2488,
          3601,  2084, 14904,  1010,  5909, 10869,  1010,  2030,  5699,  2100,
          4157,  9530,  3597, 22014,  1012,   102,     0,     0,     0,     0,
             0

In [217]:
from transformers import RobertaModel


class BERT_NER(nn.Module):

    def __init__(self, model_name, tagset_size):
        super(BERT_NER, self).__init__()
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.bert = RobertaModel.from_pretrained(model_name)
        self.lstm = nn.LSTM(input_size=1024, hidden_size=1024//2, num_layers=1, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(1024, tagset_size)
        
    def forward(self, input_ids, attention_mask, pos_tags, labels=None):
        x = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
            ).last_hidden_state
        x, _ = self.lstm(x)
        x = self.fc(x)
        return x
    
bert_ner_model = BERT_NER('bert-base-uncased', len(id2label))